In [19]:
from __future__ import division
import numpy as np
import itertools
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate

In [20]:
def coinToss(bias, numTosses): 
    # bias is always for the first element of vals
    vals = ["H", "T"]
    outcomes = list(np.random.choice(vals, numTosses, p=(bias,1 - bias)))
    #print outcomes
    # count the number of heads
    rawCount = outcomes.count("H")
    # Sometimes there can be no H at all; handle this when it happens
    try:
        normCount = rawCount/numTosses
    except ZeroDivisionError:
        normCount = 0.
    
    return rawCount, normCount

In [21]:
def probCoinTossOutcome(bias, numTosses, numEvents, slack = 0):
    
    # numEvents is the number of times we get heads when the coin is tossed numTosses times
    # slack is the range around the number of events -- when numTosses and numEvents are large
    # typically slack is a function of numTosses; it can range from 1/10 th of numTosses to 1/100 th of numTosses
    
    # Generate the outcomes
    # Keep numTrials sufficiently large but not too large - it's computationally expensive
    # 10,000 is more than sufficient; 1,000 is a good practical limit
    numTrials = 5000
    outcomes = [coinToss(bias, numTosses) for i in range(numTrials)]
    
    # Count the number of times we get numEvents heads
    # headsList = [outcome[:1] for outcome in outcomes]
    headsList = [x for (x,y) in outcomes]
    # headsCount = [h[0] for h in headsList].count(numEvents)
    # eventsFired is written to accommodate some slack when the numTosses and hence, the possible numEvents grow large
    eventsFired = [heads for heads in headsList if (heads >= numEvents - slack) and (heads <= numEvents + slack)]
    numEventsFired = len(eventsFired)
    #probOutcome = headsCount/float(numTrials)
    probOutcome = numEventsFired/float(numTrials)
    
    return [bias, numTosses, numEvents, slack, probOutcome]

In [22]:
# Normalize a list of values so that they always add up to 1
def normalize(valList):
    # valList is a list of values, e.g., [0.3, ..., 0.02]
    # valList can be any length
    sumVals = sum(valList)
    normList = []
    for val in valList:
        normVal = val/sumVals
        normList.append(normVal)
        
    return normList

In [23]:
def coinTossPValue(bias, numTosses, numEvents, slack, tailed="two-tailed"):
    
    # bias is the hypotheses that the coin has that given bias
    # numTosses is the number of times the coin is tossed
    # numEvents is the number of times we get heads when the coin is tosses numTosses times
    #  slack accounts for a range of values around numEvents rather than the exact value of numEvents
    #  slack is useful when the number of tosses is large and hence the possible numEvents is a large set
    #  In such a case, slack prevents the probability of numEvents given numTosses from always being 
    #  vanishingly close to zero
    # tailed can be "two-tailed" (default), "left-tailed" or "right-tailed"
    
    # Find the probabilities for all the possible outcomes
    probsAllOutcomes = [probCoinTossOutcome(bias, numTosses, i, slack) for i in range(numTosses + 1)]
    
    # Just get the probabilities 
    #probsList = [p[1:2][0] for p in probsAllOutcomes]
    probsList = [probs[4] for probs in probsAllOutcomes]
    
    # Normalize the probs given that the slack will produce values that will exceed 1 when added together
    normalizedProbs = normalize(probsList)
    
    # Probability of the required outcome -- i.e., numEvents
    probRequired = normalizedProbs[numEvents]
    
    # Identify and sum all the probabilities in normalizedProbs that are less than or equal to probRequired
    # including probRequired.
    # By default this is for a two-tailed sum.
    pList = []
    for p in normalizedProbs:
        if p <= probRequired:
            pList.append(p)
    
    # Left tail list of probabilities
    pListLeftTailed = []
    for index, prob in enumerate(normalizedProbs):
        if (prob <= probRequired) and (index <= numEvents):
            pListLeftTailed.append(prob)
            
    # Right tail list of probabilities
    pListRightTailed = []
    for index, prob in enumerate(normalizedProbs):
        if (prob <= probRequired) and (index >= numEvents):
            pListRightTailed.append(prob)
    
    # The two-tailed p-value is the sum of all the probabilities in pList
    pValue = sum(pList)
    
    # Left tailed p-value is the sum of all the probabilities in the pListLeftTailed
    pValueLeftTailed = sum(pListLeftTailed)
    
    # Right tailed p-value is the sum of all the probabilities in the pListRightTailed
    pValueRightTailed = sum(pListRightTailed)
    
    # Set the p-value displayed based on the input value of the tailed variable
    if tailed == "two-tailed":
        pValueDisplay = pValue
    elif tailed == "left-tailed":
        pValueDisplay = pValueLeftTailed
    elif tailed == "right-tailed":
        pValueDisplay = pValueRightTailed
    else:
        pValueDisplay = pValue
    
    #return probsList, probRequired, pList, pValue
    return [bias, numTosses, numEvents, pValueDisplay]

In [24]:
def hypoDiagnosticTest(numTosses, numEvents, alpha, slack = 0, tailed = "two-tailed"):
    # For a given bit of evidence, calculates the p-values for each possible hypothesis that 
    # could have generated that evidence. Also reveals if the hypothesis is accepted or rejected.
    
    # numTosses and numEvents together constitute the evidence
    # alpha is the significance level
    # slack is for discrete probability distributions where the x-axis values are large
    # slack is a way of treating probability mass functions with large values of numTosses 
    # as continuous functions (i.e., PDFs)
    
    # Check input integrity
    if (numEvents > numTosses) or (numEvents < 0):
        return "Please ensure that numTosses is a positive integer less than or equal to numTosses"
    
    if (alpha < 0) or (alpha > 1):
        return "Please enter a value for alpha between 0 and 1"
    
    if (slack < 0) or (slack >= numTosses):
        return "Please enter a reasonable value for the slack"
    
    if (tailed != "two-tailed") and (tailed != "left-tailed") and (tailed != "right-tailed"):
        return "Please set the tailed value to one of the following: 'two-tailed', 'left-tailed' or 'right-tailed'"
    
    # ASSUME that possible biases of the coin are 0, 0.1, ..., 1.0
    biases = list(np.arange(0,1.1,0.1))
    
    # Get the p-value for each bias hypothesis
    pValueForEachBias = [coinTossPValue(bias, numTosses, numEvents, slack, tailed) for bias in biases]
    # pValueForEachBias returns a list of lists containing [bias, numTosses, numEvents, p-value] for each bias value
    
    result = []
    for pVal in pValueForEachBias:
        if pVal[3] <= alpha:
            pVal.append("Rejected")
            result.append(pVal)
        else:
            pVal.append("Accepted")
            result.append(pVal)
            
    return result

In [25]:
def evidenceDiagnosticTest(bias, numTosses, alpha, slack = 0, tailed = "two-tailed"):
    # For a given hypothesis of the bias value, calculates the p-values for each possible bit of evidence that 
    # could be seen. Also reveals if the evidence is accepted or rejected.
    
    # numTosses and numEvents together constitute the evidence
    # numEvents is not specified explicitly because it ranges from 0 to numTosses -- 
    # i.e., every possible outcome of number of heads in numTosses
    
    # alpha is the significance level
    # slack is for discrete probability distributions where the x-axis values are large
    # slack is a way of treating probability mass functions with large values of numTosses 
    # as continuous functions (i.e., PDFs)
    
    # Check input integrity
    if (bias > 1) or (bias < 0):
        return "Please ensure that the hypoValue is a Real number between 0 and 1"
    
    if (alpha < 0) or (alpha > 1):
        return "Please enter a value for alpha between 0 and 1"
    
    if (slack < 0) or (slack >= numTosses):
        return "Please enter a reasonable value for the slack"
    
    if (tailed != "two-tailed") and (tailed != "left-tailed") and (tailed != "right-tailed"):
        return "Please set the tailed value to one of the following: 'two-tailed', 'left-tailed' or 'right-tailed'"
    
    # Generate all possible events
    events = range(numTosses + 1)
    
    # Get the p-value for each bias hypothesis
    pValueForEachEvent = [coinTossPValue(bias, numTosses, event, slack, tailed) for event in events]
    # pValueForEachEvent returns a list of lists containing [bias, numTosses, numEvents, p-value] for each event value
    
    result = []
    for eVal in pValueForEachEvent:
        if eVal[3] <= alpha:
            eVal.append("Rejected")
            result.append(eVal)
        else:
            eVal.append("Accepted")
            result.append(eVal)
            
    return result

In [26]:
# NOT USED
# A function that perturbs a value
# For a given set of possibilities, generate a set of probabilities 
def perturb(val, min, max, midPoint):
    # val is the value that is being perturbated
    # min is the minimum amount of perturbation
    # max is the maximum amount of perturbation
    # midPoint is the mid point between min and max. It can be set symmetrically
    # or asymmetrically.
    
    # Check that midPoint is between min and max
    if (midPoint < min) or (midPoint > max):
        return "Please choose a midPoint between min and max"
        
    
    pertVal = val + np.random.triangular(min,midPoint,max)
    
    return pertVal

In [27]:
# Generate a random list of reals of a given length that add up to 1
def discreteRandomDist(possibleOutcomes):
    # possibleOutcomes are all the discrete events that are possible
    # e.g., range(11) or ["a","b","c"]
    
    # generate reals between 0 and 1 (number of reals = length(possibleOutcomes))
    probs = np.random.random_sample(len(possibleOutcomes))
    
    # normalize the probabilities to sum to 1
    normalizedProbs = normalize(probs)
   
    return zip(possibleOutcomes, normalizedProbs)

In [28]:
discreteRandomDist(range(11))

[(0, 0.093566419425087546),
 (1, 0.087056770276490045),
 (2, 0.02358088158193759),
 (3, 0.03700801839715348),
 (4, 0.12117298601488767),
 (5, 0.15853615466541648),
 (6, 0.11379411706656661),
 (7, 0.069036280721851181),
 (8, 0.14455597570060991),
 (9, 0.028135937145045656),
 (10, 0.12355645900495391)]

In [29]:
# A function that calculates the probability of a hypothesis given some evidence.
# The hypothesis is in the form of a bias value for a coin
# The priors are randomly generated
# The evidence is in the form of the number of heads observed when the coin is tosses N times.
# WORKS ONLY FOR DISCRETE OUTCOME STATES BETWEEN 0 and 1 IN INCREMENTS OF 0.1

def bayes(bias, numTosses, numEvents, slack = 0):
    
    # generate a random initial distribution for all possible biases of the coin
    # restricting the allowed biases to the Reals 0, 0.1, 0.2, ..., 0.9, 1.
    # priorDist is the prior distribution over the possible bias values for the coin.
    priorDist = discreteRandomDist(np.arange(0,1.1,0.1))
    # just get the values
    priorDistVals = [y for (x,y) in priorDist]
    
    # calculate the probability of the evidence for each possible bias value of the coin
    probEvidenceForEachBias = [probCoinTossOutcome(i,numTosses,numEvents, slack) for i in np.arange(0,1.1,0.1)]
    # just get the values and normalize them in case slack is > 0
    probEvidenceForEachBiasVals = normalize([probEvidence[4] for probEvidence in probEvidenceForEachBias])
    
    # Probability of the evidence per se - i.e., the Bayesian normalization factor
    pEvidence = np.sum(np.multiply(priorDistVals, probEvidenceForEachBiasVals))
    
    # From the priorDistVals, find the prior for the particular bias
    # CAUTION: ONLY WORKS FOR DISCRETE COIN BIASES FROM 0 TO 1 in intervals of 0.1
    biasPrior = priorDistVals[int(bias * 10)]
    
    # From the probEvidenceForAllHyposVals, find the probEvidence for the particular bias
    # CAUTION: ONLY WORKS FOR DISCRETE COIN BIASES FROM 0 TO 1 in intervals of 0.1
    probEvidenceForBias = probEvidenceForEachBiasVals[int(bias * 10)]
    
    # Finally, find the probability for that the bias is what it is, given the evidence
    bayesPBias = (biasPrior * probEvidenceForBias)/pEvidence
    
    
    return biasPrior, bayesPBias

In [30]:
# bayesFull calculates the posterior probabilities
# of a set of hypotheses. This allows us to compare the prior with the posterior
# distribution for all hypotheses
# WORKS ONLY FOR DISCRETE SETS OF HYPO VALUES

def bayesFull(hyposAndPriorProbs, numTosses, numEvents, slack = 0):
    # hypos is the set of biases, e.g., [0, 0.1, ..., 0.9, 1.0] for the possible discrete biases of a coin
    # priorProbs is the prior probability of each of the hypos
    
    # We get hyposAndPriorProbs as the output of a function such as discreteRandomDist(np.arange(0,1.1,0.1))
    # The evidence is in the form of the number of heads observed (numEvents) 
    # when the coin is tosses N times (numTosses).
    
    # WORKS ONLY FOR DISCRETE SETS OF HYPO VALUES
    
    # Split hyposAndPriorProbs into hypos and priorProbs
    hypos = [hyposAndPriorProb[0] for hyposAndPriorProb in hyposAndPriorProbs]
    priorProbs = [hyposAndPriorProb[1] for hyposAndPriorProb in hyposAndPriorProbs]
    
    # calculate the probability of the evidence for each possible hypo
    probEvidenceForEachHypo = [probCoinTossOutcome(hypo,numTosses,numEvents, slack) for hypo in hypos]
    # just get the values and normalize in case slack > 0
    probEvidenceForEachHypoVals = normalize([probEvidence[4] for probEvidence in probEvidenceForEachHypo])
    
    # Probability of the evidence per se - i.e., the Bayesian normalization factor
    pEvidencePerSe = np.sum(np.multiply(priorProbs, probEvidenceForEachHypoVals))
    
    # Posterior distribution without Bayesian normalization
    postDistRaw = list(np.multiply(priorProbs, probEvidenceForEachHypoVals))
    
    # Bayesian normalized posterior distribution -- this is usually the one we want
    postDist = list(postDistRaw/pEvidencePerSe)
    
    return priorProbs, postDist

In [31]:
# Symbolic partial differentiation -- examples
from sympy import symbols, diff
x, y, z = symbols('x y z', real=True)
f = 4*x*y + x**3 + z**8*y
diff(f, x)

3*x**2 + 4*y

In [32]:
theta0, theta1, theta2, theta3, theta4, x0, x1 = symbols('theta0, theta1, theta2, theta3, theta4, x0, x1', real=True)
g = theta0*x0 + theta1*x1 + theta2*x1**2 + theta3*x1**3 + theta4*x1**(2.5)
diff(g, theta4)

x1**2.5

In [33]:
# Take a data set and split it into training, cross-validation, and test data sets
def splitDataSet(data, trainRatio, valRatio):
    # Randomly shuffle the data -- 
    # Split up into train (trainRatio of all data), cross validate(valRatio), and test data sets
    # NOTE - there's also a sklearn function train_test_split to do a similar thing but not quite
    dataArr = np.array(data)
    dataDim = dataArr.shape
    #print dataArr[0:4,:]
    np.random.shuffle(dataArr)
    #print dataArr[0:4,:]

    # Find out the number of data rows so they can be split up in the right proportion
    num_train_rows = int(dataDim[0] * trainRatio)
    num_val_rows = int(dataDim[0] * valRatio)
    num_test_rows = dataDim[0] - (num_train_rows + num_val_rows)

    # Training Data
    X_train = dataArr[0:num_train_rows, 0].reshape((num_train_rows, 1))
    y_train = dataArr[0:num_train_rows, 1].reshape((num_train_rows, 1))

    # Cross Validation Data
    X_val = dataArr[num_train_rows:num_train_rows + num_val_rows, 0].reshape((num_val_rows, 1))
    y_val = dataArr[num_train_rows:num_train_rows + num_val_rows, 1].reshape((num_val_rows, 1))

    # Test Data
    X_test = dataArr[:num_test_rows, 0].reshape((num_test_rows, 1))
    y_test = dataArr[:num_test_rows, 1].reshape((num_test_rows, 1))
    
    return [X_train, y_train, X_val, y_val, X_test, y_test]

In [34]:
def computeCost(X, y, theta):
    # Cost of being wrong calculated over the entire data set
    # Assumes that X has a first column of 1s added to it to simplify the notation
    # Therefore: X is an m x n matrix and theta is a n x 1 matrix
    
    # costPerOutput is an m x 1 matrix where each element is the cost for
    # the input and its associated output for a particular value of theta
    costPerOutput = np.power(((X * theta) - y),2)
    
    # totalCost is the cost over the entire dataset
    totalCost = np.sum(costPerOutput)
    
    # The cost of getting it wrong is 1/2m of the totalCost (normalized cost)
    cost = totalCost / (2 * len(X))
    
    return cost

In [36]:
# Implement Gradient Descent
def gradientDescent(X, y, theta, alpha, iters):
    # NOTE: X must already have a column of 1s added to it
    # X is a m x n matrix
    # y is a m x 1 matrix
    # Theta is a n x 1 matrix
    
    # Keep track of everything
    sumError = np.zeros(shape=(len(theta),1))
    sumErrorNorm = np.zeros(shape=(len(theta),1))
    temp = np.matrix(np.zeros(theta.shape))
    cost = np.zeros(iters)
    
    for i in range(iters):
        # Calculate the non-normalized values for each theta parameter
        error = (X * theta) - y
        
        for j in range(len(theta)):
            # Multiply the error vector by the appropriate column of the X matrix and sum it
            sumError[j] = np.sum(np.multiply(error, X[:,j]))
            
            # Normalize the sumError using alpha and m
            sumErrorNorm[j] = np.divide(np.multiply(sumError[j], alpha), len(X))
            
            temp[j,0] = theta[j,0] - sumErrorNorm[j]
        
        theta = temp
        cost[i] = computeCost(X,y,theta)
            
    
    return theta, cost